In [1]:
import pandas as pd
from Crypto.Cipher import AES
from PIL import Image
import io

input_csv = "iladata.csv"
key = bytes.fromhex('000102030405060708090a0b0c0d0e0f')
delay_cycles=12

In [2]:
data_columns=pd.read_csv(input_csv)
data_columns=data_columns[["addra[1][6:0]","TRIGGER","done","encrypted128[0][127:0]","encrypted128[1][127:0]","encrypted128[2][127:0]","encrypted128[3][127:0]","encrypted128[4][127:0]","encrypted128[5][127:0]"]]
data_columns = data_columns.drop(index=0).reset_index(drop=True)

columns_to_convert = ["addra[1][6:0]", "TRIGGER", "done"]

for col in columns_to_convert:
    data_columns[col] = data_columns[col].astype(int)

trigger_index = data_columns[data_columns["TRIGGER"] == 1].index[0]
data_columns = data_columns.iloc[trigger_index+delay_cycles:].reset_index(drop=True)

done_index=data_columns[data_columns["done"] == 1].index[0]
data_columns = data_columns.iloc[:done_index+delay_cycles].reset_index(drop=True)
data_columns

KeyError: "['addra[1][6:0]', 'TRIGGER', 'encrypted128[0][127:0]', 'encrypted128[1][127:0]', 'encrypted128[2][127:0]', 'encrypted128[3][127:0]', 'encrypted128[4][127:0]', 'encrypted128[5][127:0]'] not in index"

In [ ]:
# Step 1: Reverse the second round-robin split
iterate_col=["encrypted128[0][127:0]","encrypted128[1][127:0]","encrypted128[2][127:0]","encrypted128[3][127:0]","encrypted128[4][127:0]","encrypted128[5][127:0]"]
recombined_data_stage1 = [[] for _ in range(3)]  # Specify the column names here
for j in range(3):
    for i in range(len(data_columns["encrypted128[0][127:0]"])):
        recombined_data_stage1[j].append(data_columns[iterate_col[j*2]][i])
        recombined_data_stage1[j].append(data_columns[iterate_col[j*2+1]][i])

final_data = []

for i in range(len(recombined_data_stage1[0])):
    final_data.append(recombined_data_stage1[0][i])
    final_data.append(recombined_data_stage1[2][i])
    final_data.append(recombined_data_stage1[1][i])

df = pd.DataFrame(final_data)
df.columns = ["Encrypted_image"]
df

,Encrypted_image
0,9958fdec8e2d65c80160cfc0e66432fa
1,defc899e94c6b8ce8fedcee9e9ff3dcf
2,e109ac33e298773a8a915b24c78e054f
3,5d8b8af9f919c8e6dfc310d44dc36c19
4,9292c41266e1f4e3e689ec80dfb602fe
...,...
271,c6a13b37878f5b826f4f8162a1c8d879
272,c6a13b37878f5b826f4f8162a1c8d879
273,c6a13b37878f5b826f4f8162a1c8d879
274,c6a13b37878f5b826f4f8162a1c8d879


In [ ]:
def decrypt_data(encrypted_hex, key):
    try:
        # Convert the encrypted text from hex to bytes
        encrypted_data = bytes.fromhex(encrypted_hex)
        
        if len(encrypted_data) != 16:
            raise ValueError("Encrypted data must be 128 bits (16 bytes)")

        # Create a new AES cipher in ECB mode
        cipher = AES.new(key, AES.MODE_ECB)
        
        # Decrypt the data
        decrypted_data = cipher.decrypt(encrypted_data)        
        # Return decrypted data in hexadecimal format
        return decrypted_data.hex()
    except ValueError as e:
        return f"Error: {e}"

In [ ]:
dec_df=pd.DataFrame()
dec_df['decrypted_col'] = df['Encrypted_image'].apply(lambda x: decrypt_data(x, key))
dec_df

,decrypted_col
0,ffd8ffe000104a464946000101000001
1,00010000ffdb00430008060607060508
2,0707070909080a0c140d0c0b0b0c1912
3,130f141d1a1f1e1d1a1c1c20242e2720
4,222c231c1c2837292c30313434341f27
...,...
271,00000000000000000000000000000000
272,00000000000000000000000000000000
273,00000000000000000000000000000000
274,00000000000000000000000000000000


In [ ]:
# Convert the hexadecimal decrypted data to binary and write it to a .bin file
with open("decrypted_image.bin", "wb") as bin_file:
    for hex_string in dec_df['decrypted_col']:
        # Only process valid hex strings
        if hex_string and "Error" not in hex_string:
            binary_data = bytes.fromhex(hex_string)
            # Write binary data to file
            bin_file.write(binary_data)

with open("encrypted_image.bin", "wb") as bin_file:
    for hex_string in df['Encrypted_image']:
        # Only process valid hex strings
        if hex_string and "Error" not in hex_string:
            # Strip any padding spaces or newlines from hex strings
            binary_data = bytes.fromhex(hex_string)
            # Write binary data to file
            bin_file.write(binary_data)


In [ ]:
def display_image_from_binary(binary_file_path,image_path):
    try:
        # Read the binary data from the file
        with open(binary_file_path, 'rb') as binary_file:
            img_binary_data = binary_file.read()
        img = Image.open(io.BytesIO(img_binary_data))
        
        img.show()
        img.save(image_path)

    except Exception as e:
        print(f"Error displaying image: {e}")

display_image_from_binary("decrypted_image.bin","decrypted_image.jpeg")
display_image_from_binary("encrypted_image.bin","encrypted_image.jpeg")

Error displaying image: cannot identify image file <_io.BytesIO object at 0x76df355b6f20>
